In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import scipy.sparse as sp
import sympy as spsy
import numpy as np
from sympy import symbols, interpolate, log

In [2]:
def binary_string_to_graph(binary_string):
    n = len(binary_string)
    row_indices = []
    col_indices = []

    for i in range(n):
        if binary_string[i] == '0':
            for j in range(i + 1, n):
                row_indices.append(i)
                col_indices.append(j)
        elif binary_string[i] == '1':
            for j in range(i):
                row_indices.append(i)
                col_indices.append(j)

    graph = sp.csr_matrix((n, n), dtype=int)
    graph[row_indices, col_indices] = 1

    return graph

In [3]:
def visualize_graph(graph):
    nx_graph = nx.Graph(graph)

    pos = nx.spring_layout(nx_graph)
    nx.draw(nx_graph, pos, with_labels=True, node_color='lightblue', node_size=500, font_size=10, font_weight='bold', width=0.5)

    plt.title("Graph Visualization")
    plt.axis('off')
    plt.show()

In [4]:
def create_binary_matrix(graph, n):
    binary_matrix = sp.lil_matrix((n, n), dtype=int)
    row_indices, col_indices = graph.nonzero()
    for i, j in zip(row_indices, col_indices):
        binary_matrix[i, j] = 1
    return binary_matrix

In [5]:
def binary_to_decimal(binary_row):
    decimal_value = 0
    for bit in binary_row:
        decimal_value = (decimal_value << 1) | bit
    return decimal_value

In [6]:
def retrieve_binary_string(binary_matrix):
    n = binary_matrix.shape[0]
    binary_string = ""

    # Handle the first row
    if all(binary_matrix[0, j] == 0 for j in range(n)):
        binary_string += '1'

    i = 0
    while i < n:
        found = False
        for j in range(i + 1):
            if binary_matrix[i, j] == 1:
                binary_string += '1'
                found = True
                break
        if not found:
            for j in range(i + 1, n):
                if binary_matrix[i, j] == 1:
                    binary_string += '0'
                    break
        i += 1

    # Handle the last row
    if all(binary_matrix[n-1, j] == 0 for j in range(n)):
        binary_string += '0'

    return binary_string

In [7]:
def binary_matrix_to_decimal_list(binary_matrix):
    n = binary_matrix.shape[0]
    decimal_list = []
    for j in range(n):
        decimal_value = 0
        for i in range(n):
            decimal_value += binary_matrix[i, j] * (2 ** i)
        decimal_list.append(decimal_value)
    return decimal_list

In [8]:
def generate_chromatic_polynomial(decimal_list):
    x = spsy.symbols('x')
    chromatic_polynomial = 0
    n = len(decimal_list)

    for i, coeff in enumerate(decimal_list):
        chromatic_polynomial += coeff * (x ** (n - (i+1)))

    return chromatic_polynomial

In [9]:
def calculate_chromatic_values(chromatic_polynomial, n, r):
    chromatic_values = []
    for x in range(r, n + r):
        y = chromatic_polynomial.subs('x', x)
        chromatic_values.append(y)
    return chromatic_values

In [12]:
binary_string = input("Enter an n-bit binary string: ")
graph = binary_string_to_graph(binary_string)
#visualize_graph(graph)

r = int(input("Enter the value of r: "))

n = graph.shape[0]
binary_matrix = create_binary_matrix(graph, n)
print("Binary Matrix:")
print(binary_matrix.toarray())

retrieved_binary_string = retrieve_binary_string(binary_matrix)
print("Retrieved Binary String:", retrieved_binary_string)

decimal_list = binary_matrix_to_decimal_list(binary_matrix)
print("Decimal Values of Columns:", decimal_list)

chromatic_polynomial = generate_chromatic_polynomial(decimal_list)
print("Chromatic Polynomial:", chromatic_polynomial)

chromatic_values = calculate_chromatic_values(chromatic_polynomial, n, r)
print("Chromatic Polynomial Values: ", chromatic_values)

Enter an n-bit binary string: 101101
Enter the value of r: 7
Binary Matrix:
[[0 0 0 0 0 0]
 [0 0 1 1 1 1]
 [1 1 0 0 0 0]
 [1 1 1 0 0 0]
 [0 0 0 0 0 1]
 [1 1 1 1 1 0]]
Retrieved Binary String: 101101
Decimal Values of Columns: [44, 44, 42, 34, 34, 18]
Chromatic Polynomial: 44*x**5 + 44*x**4 + 42*x**3 + 34*x**2 + 34*x + 18
Chromatic Polynomial Values:  [861480, 1645986, 2920536, 4885758, 7790856, 11938890]
